In [1]:
import os
import tqdm
os.chdir("/home/YiVal_test/")

In [11]:
! git clone https://github.com/NVlabs/stylegan3.git
! git clone https://github.com/NVlabs/metfaces-dataset.git

fatal: destination path 'stylegan3' already exists and is not an empty directory.


Cloning into 'metfaces-dataset'...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 14 (delta 5), reused 12 (delta 3), pack-reused 0
Unpacking objects: 100% (14/14), 623.34 KiB | 5.28 MiB/s, done.


In [3]:
! python -V
! pip -V

Python 3.10.13


pip 23.3.1 from /home/YiVal_test/.cache/pypoetry/virtualenvs/yival-0EQAwXAc-py3.10/lib/python3.10/site-packages/pip (python 3.10)


In [4]:
! pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.33.0 trl==0.4.7 scipy tensorboard tensorboardX tqdm ipywidgets bitsandbytes-cuda117 sentencepiece


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [5]:
! pip install -q click Pillow gdown Ninja torch torchvision torchaudio


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [6]:
! pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118, https://pypi.ngc.nvidia.com


In [12]:
! python /home/YiVal_test/metfaces-dataset/metfaces.py --json /share/data_shared/metfaces-release/metfaces-dataset.json --source-images /share/data_shared/metfaces-release \
    --random-shift=0.2 --retry-crops --no-rotation \
    --output-dir /home/YiVal_test/metfacesu

  0%|                                                  | 0/1336 [00:00<?, ?it/s]/home/YiVal_test/metfaces-dataset/metfaces.py:126: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  img = img.resize((target_size, target_size), PIL.Image.ANTIALIAS)
100%|███████████████████████████████████████| 1336/1336 [28:07<00:00,  1.26s/it]


In [2]:
os.chdir("/home/YiVal_test/stylegan3/")
#! export 'PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:256'
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128"

In [5]:
! python /home/YiVal_test/stylegan3/train.py --outdir=~/training-runs --cfg=stylegan3-r --data=/home/YiVal_test/metfacesu \
--gpus=1 --batch=32 --batch-gpu=4 --gamma=6.6 --mirror=1 --kimg=5000 --snap=5 --resume=https://api.ngc.nvidia.com/v2/models/nvidia/research/stylegan3/versions/1/files/stylegan3-r-ffhqu-1024x1024.pkl


Training options:
{
  "G_kwargs": {
    "class_name": "training.networks_stylegan3.Generator",
    "z_dim": 512,
    "w_dim": 512,
    "mapping_kwargs": {
      "num_layers": 2
    },
    "channel_base": 65536,
    "channel_max": 1024,
    "magnitude_ema_beta": 0.9988915792636801,
    "conv_kernel": 1,
    "use_radial_filters": true
  },
  "D_kwargs": {
    "class_name": "training.networks_stylegan2.Discriminator",
    "block_kwargs": {
      "freeze_layers": 0
    },
    "mapping_kwargs": {},
    "epilogue_kwargs": {
      "mbstd_group_size": 4
    },
    "channel_base": 32768,
    "channel_max": 512
  },
  "G_opt_kwargs": {
    "class_name": "torch.optim.Adam",
    "betas": [
      0,
      0.99
    ],
    "eps": 1e-08,
    "lr": 0.0025
  },
  "D_opt_kwargs": {
    "class_name": "torch.optim.Adam",
    "betas": [
      0,
      0.99
    ],
    "eps": 1e-08,
    "lr": 0.002
  },
  "loss_kwargs": {
    "class_name": "training.loss.StyleGAN2Loss",
    "r1_gamma": 6.6,
    "blur_init_si